In [1]:
from pathlib import Path
import pandas as pd
import csv

data_path = Path.cwd().parent / "data"

In [2]:
# EJSCreen csv Load
ejscreen_csv = data_path / "dataset" / "ejscreen_2020" / "usa.csv"
df = pd.read_csv(ejscreen_csv, dtype={'ID': 'string'}, low_memory=False)
df.head()

,ID,ACSTOTPOP,LESSHSPCT,LOWINCPCT
0,010010201001,636,0.208134,0.385220
1,010010201002,1287,0.040678,0.163170
2,010010202001,810,0.135563,0.501247
3,010010202002,1218,0.192000,0.393701
4,010010203001,2641,0.125473,0.308217


In [5]:
# calculate percentiles
df['lesshs_percentile'] = df.LESSHSPCT.rank(pct = True)
df['lowin_percentile'] = df.LOWINCPCT.rank(pct = True)
df['score_a'] = df[['lesshs_percentile', 'lowin_percentile']].mean(axis=1)
df['score_b'] = df.lesshs_percentile * df.lowin_percentile
df.head()

,ID,ACSTOTPOP,LESSHSPCT,LOWINCPCT,lesshs_percentile,lowin_percentile,score_a,score_b
0,010010201001,636,0.208134,0.385220,0.793292,0.625015,0.709154,0.495820
1,010010201002,1287,0.040678,0.163170,0.238550,0.246722,0.242636,0.058856
2,010010202001,810,0.135563,0.501247,0.634390,0.772002,0.703196,0.489750
3,010010202002,1218,0.192000,0.393701,0.765126,0.637158,0.701142,0.487506
4,010010203001,2641,0.125473,0.308217,0.603841,0.504977,0.554409,0.304925
